# FLEXible tutorial: Two types of updating weights using Pytorch

FLEXible is a library to federate models. We offer the tools to load and federate data or to load federated data, and the tools to create a federated environment. The user can define the model and the *communications primitives* to train the model in a federated environment, but we already offer decorators so that an advaced user can implement its own federated workflow. In addition, we offer some primitive functions for Pytorch. Those primitives are basics functions, as we assume how the federated learning training will be. If you want to a more customizable training loop, please check the notebook "Federated MNIST PT example with flexible decorators", as we show there how to implement the primitive functions from scrach. 

In this notebook, we show the two types of updating the weights of the client's model in federated learning. These methods are:
- Updating the weights from the local models for the global weights recieved.
- Updating the local weights by adding the difference between the global weights recieved and the local weights from the last round.

In [1]:
import torch
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
device

'mps'

First load the data, in this case we've chosen the MNIST dataset. This dataset is directy plugged onto FLEXible, so we can use the load function to get it directly federated.

In [2]:
from flex.datasets import load
from torchvision import transforms

flex_dataset, test_data = load("federated_emnist", return_test=True, split="digits")

mnist_transforms = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize((0.5,), (0.5,))
        ])

[sultan]: md5 -q ./emnist-digits.mat;
[sultan]: md5 -q ./emnist-digits.mat;


`@init_server_model` is a decorator designed to perform the initialization of the server model in a client-server architecture. It has no requirements for specific arguments in the function that uses it.

In [3]:
import torch.nn as nn
import torch.nn.functional as F

from flex.pool import init_server_model
from flex.pool import FlexPool
from flex.model import FlexModel

class SimpleNet(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(28*28, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

@init_server_model
def build_server_model():
    server_flex_model = FlexModel()

    server_flex_model["model"] = SimpleNet().to(device) # Ensure all models are on the same device
    # Required to store this for later stages of the FL training process
    server_flex_model["criterion"] = torch.nn.CrossEntropyLoss()
    server_flex_model["optimizer_func"] = torch.optim.Adam
    server_flex_model["optimizer_kwargs"] = {}

    return server_flex_model

flex_pool = FlexPool.client_server_architecture(flex_dataset, init_func=build_server_model)

clients = flex_pool.clients
servers = flex_pool.servers
aggregators = flex_pool.aggregators

print(f"Number of nodes in the pool {len(flex_pool)}: {len(servers)} server plus {len(clients)} clients. The server is also an aggregator")

Number of nodes in the pool 3580: 1 server plus 3579 clients. The server is also an aggregator


We also implement the possibility of select a subsample of the clients in the training process.

In [4]:
#Select clients
clients_per_round=2
selected_clients_pool = clients.select(clients_per_round)
selected_clients = selected_clients_pool.clients

print(f"Server node is indentified by key \"{servers.actor_ids[0]}\"")
print(f"Selected {len(selected_clients.actor_ids)} client nodes of a total of {len(clients.actor_ids)}")

Server node is indentified by key "server"
Selected 2 client nodes of a total of 3579


`@deploy_server_model` is a decorator designed to copy the model from the server to the clients at each federated learning round. The function that uses it, must have at least one argument, which is the FlexModel object that stores the model at the server.

In [5]:
from flex.pool.primitives_pt import deploy_server_model_pt

servers.map(deploy_server_model_pt, selected_clients)

## Two ways of updateing the weights of the client's model.

Suprisingly, there is no decorator for the training process as it can be imnplemented directly. In this notebook we present primitives for both ways of updating the weights when using Pytorch, so the user can use them without needing to implement a new one, but if a different way of updating the weights it's needed, it can be created using the decorators for collecting and sendings weights. For more info check the notebook **Federated MNIST PT example with flexible decorators**, where we show how to create your own primitives.

### Updating the weights from the local models for the global weights recieved.

First we show how to update the weights by just changing the weights by the new ones.

In [6]:
from flex.data import Dataset
from torch.utils.data import DataLoader
from copy import deepcopy
from tqdm import tqdm

def train(client_flex_model: FlexModel, client_data: Dataset):
    train_dataset = client_data.to_torchvision_dataset(transform=mnist_transforms)
    client_dataloader = DataLoader(train_dataset, batch_size=64)
    model = client_flex_model["model"]
    # client_flex_model['previous_model'] = deepcopy(model) # Required to use `collect_client_diff_weights_pt` primitive
    optimizer = client_flex_model['optimizer_func'](model.parameters(), **client_flex_model["optimizer_kwargs"])
    model = model.train()
    model = model.to(device)
    criterion = client_flex_model["criterion"]
    epochs = 5
    for _ in tqdm(range(epochs)):
        for imgs, labels in client_dataloader:
            imgs, labels = imgs.to(device), labels.to(device)
            optimizer.zero_grad()
            pred = model(imgs)
            loss = criterion(pred, labels)
            loss.backward()
            optimizer.step()

In [7]:
selected_clients.map(train)

100%|██████████| 5/5 [00:00<00:00, 51.96it/s]


The primitives for updating the weights by just changing one weights for other are `collect_clients_weights_pt` and `set_aggregated_weights_pt`.

In [8]:
from flex.pool import collect_clients_weights_pt, set_aggregated_weights_pt

aggregators.map(collect_clients_weights_pt, selected_clients)

`fed_avg` implements the aggregator Fedeverated Average, which computes the mean of the collected weights in previous steps.

In [9]:
from flex.pool import fed_avg

aggregators.map(fed_avg)

Once the weights are aggregated, we can send them to the clients

In [10]:
aggregators.map(set_aggregated_weights_pt, servers)

`@evaluate_server_model` is a decorator used to test the server model. The function that uses it must have at least one argument, the FlexModel at the server.

In [11]:
from flex.pool import evaluate_server_model

@evaluate_server_model
def evaluate_global_model(server_flex_model: FlexModel, test_data=None):
    model = server_flex_model["model"]
    model.eval()
    test_loss = 0
    test_acc = 0
    total_count = 0
    model = model.to(device)
    criterion=server_flex_model['criterion']
    # get test data as a torchvision object
    test_dataset = test_data.to_torchvision_dataset(transform=mnist_transforms)
    test_dataloader = DataLoader(test_dataset, batch_size=256, shuffle=True, pin_memory=False)
    losses = []
    with torch.no_grad():
        for data, target in test_dataloader:
            total_count += target.size(0)
            data, target = data.to(device), target.to(device)
            output = model(data)
            losses.append(criterion(output, target).item())
            pred = output.data.max(1, keepdim=True)[1]
            test_acc += pred.eq(target.data.view_as(pred)).long().cpu().sum().item()

    test_loss = sum(losses) / len(losses)
    test_acc /= total_count
    return test_loss, test_acc

In [12]:
metrics = servers.map(evaluate_global_model, test_data=test_data)
print(metrics[0])

(2.1045245008104168, 0.136775)


### Updating the weights by calculating the difference between the global and the local model.

Lastly, we show the process on how to update the weights by calculating the diference between the global model and the local model of the clients. To do so, we need to add a new key to the client's FlexModel, *previous_model*, to copy the global model recieved by the last round.

In [13]:
from flex.data import Dataset
from torch.utils.data import DataLoader
from copy import deepcopy
from tqdm import tqdm

def train(client_flex_model: FlexModel, client_data: Dataset):
    train_dataset = client_data.to_torchvision_dataset(transform=mnist_transforms)
    client_dataloader = DataLoader(train_dataset, batch_size=64)
    model = client_flex_model["model"].to(device)
    client_flex_model['previous_model'] = deepcopy(model) # Required to use `collect_client_diff_weights_pt` primitive
    optimizer = client_flex_model['optimizer_func'](model.parameters(), **client_flex_model["optimizer_kwargs"])
    model = model.train()
    criterion = client_flex_model["criterion"]
    epochs = 5
    for _ in tqdm(range(epochs)):
        for imgs, labels in client_dataloader:
            imgs, labels = imgs.to(device), labels.to(device)
            optimizer.zero_grad()
            pred = model(imgs)
            loss = criterion(pred, labels)
            loss.backward()
            optimizer.step()

In [14]:
selected_clients.map(train)

100%|██████████| 5/5 [00:00<00:00, 119.28it/s]


`collect_client_diff_weights_pt` as it name says, it collects weights from a set of clients. Particularly, it collects the difference between the model before and after training, that is, what the model has learnt in its local training step. Also note that the weights of the model before training are assume to be stored using `previous_model` as key in the FlexModel of a client.

In [15]:
from flex.pool import collect_client_diff_weights_pt

aggregators.map(collect_client_diff_weights_pt, selected_clients)

We also use the `fed_avg` aggregator.

In [16]:
from flex.pool import fed_avg

aggregators.map(fed_avg)

`set_aggregated_diff_weights_pt` adds the aggregated weights to the weights of the server, it assumes that the aggregated weights have been collected using a similar logic to `collect_client_diff_weights_pt`.

In [17]:
from flex.pool import set_aggregated_diff_weights_pt

aggregators.map(set_aggregated_diff_weights_pt, servers)

To evaluate the server model, we use the same function created in the last section.

In [18]:

metrics = servers.map(evaluate_global_model, test_data=test_data)
print(metrics[0])

(1.590213431674204, 0.5245)


### Run the federated learning experiment for a few rounds

Now, we can summarize the steps provided above and run the federated experiment for multiple rounds:

In [19]:
# Auxiliar function to clear unused gpu mem in clients
def clean_up_models(client_model: FlexModel, _):
    import gc
    del client_model["model"]
    del client_model["previous_model"]
    gc.collect()

def train_n_rounds(n_rounds, clients_per_round=20):  
    pool = FlexPool.client_server_architecture(fed_dataset=flex_dataset, init_func=build_server_model)
    for i in range(n_rounds):
        print(f"\nRunning round: {i+1} of {n_rounds}")
        selected_clients_pool = pool.clients.select(clients_per_round)
        selected_clients = selected_clients_pool.clients
        print(f"Selected clients for this round: {len(selected_clients)}")
        # Deploy the server model to the selected clients
        pool.servers.map(deploy_server_model_pt, selected_clients)
        # Each selected client trains her model
        selected_clients.map(train)
        # The aggregador collects weights from the selected clients and aggregates them
        pool.aggregators.map(collect_client_diff_weights_pt, selected_clients)
        pool.aggregators.map(fed_avg)
        # The aggregator send its aggregated weights to the server
        pool.aggregators.map(set_aggregated_diff_weights_pt, pool.servers)
        # Optional: evaluate the server model
        metrics = pool.servers.map(evaluate_global_model, test_data=test_data)
        # Optional: clean-up unused memory
        selected_clients.map(clean_up_models)
        loss, acc = metrics[0]
        print(f"Server: Test acc: {acc:.4f}, test loss: {loss:.4f}")

In [21]:
train_n_rounds(20, clients_per_round=10)


Running round: 1 of 20
Selected clients for this round: 10


  0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 5/5 [00:00<00:00, 57.01it/s]


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, mps:0 and cpu!